<a href="https://colab.research.google.com/github/Dinal-Senadheera/simpleNLP/blob/main/simpleNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import dependencies

In [ ]:
import pandas as pd

Reading the data

In [ ]:
df = pd.read_csv('Tweets.csv')
df.shape

(14640, 15)

In [ ]:
df.shape

(14640, 15)

Getting required columns

In [ ]:
df = df[["airline_sentiment", "text"]]
df

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


importing dependecies need for preprocessing the text

In [ ]:
import nltk
import string
import re

from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords


nltk.download('punkt')
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Preprocessing text

In [ ]:
def clean_text(text) :
  text = text.lower()
  text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
  text = nltk.word_tokenize(text)

  y = []

  for i in text:
    if i not in stopwords.words('english'):
      y.append(i)

  text = y[:]
  y.clear()

  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)

Creating a col with preprocessed text

In [ ]:
df['text_cleaned'] = df['text'].apply(clean_text)

In [ ]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9178   9087   
neutral            3099   3067   
positive           2363   2298   

                                                                           \
                                                                 top freq   
airline_sentiment                                                           
negative           @AmericanAir that's 16+ extra hours of travel ...    2   
neutral                                           @SouthwestAir sent    5   
positive                                            @JetBlue thanks!    5   

                  text_cleaned         \
                         count unique   
airline_sentiment                       
negative                  9178   9083   
neutral                   3099   3025   
positive                  2363   2262   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @ americanair fyi ... call still get drop . ho...    2  
neutral            @ jetblu 's ceo battl appeas passeng wall stre...    8  
positive                                            @ jetblu thank !   12

In [ ]:
df = df.drop_duplicates(keep = 'first')

In [ ]:
df.shape

(14452, 3)

In [ ]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9087   9087   
neutral            3067   3067   
positive           2298   2298   

                                                                           \
                                                                 top freq   
airline_sentiment                                                           
negative           @VirginAmerica it's really aggressive to blast...    1   
neutral                          @VirginAmerica What @dhepburn said.    1   
positive           @VirginAmerica plus you've added commercials t...    1   

                  text_cleaned         \
                         count unique   
airline_sentiment                       
negative                  9087   9083   
neutral                   3067   3025   
positive                  2298   2262   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @ unit airlin cancel flight oc fll # flight to...    2  
neutral            @ jetblu 's ceo battl appeas passeng wall stre...    8  
positive                                              @ unit thank !    5

Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf_vec = TfidfVectorizer(max_features = 3000)
X = tf_vec.fit_transform(df['text_cleaned']).toarray()

In [ ]:
X.shape

(14452, 3000)

In [ ]:
Y = df['airline_sentiment'].values

In [ ]:
Y.shape

(14452,)

Splitting data

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2, random_state = 2)

Creating and training model

In [ ]:
#multinomial model
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)
print(accuracy_score(Y_test,Y_pred))

0.7391905914908337


In [ ]:
#Random Forest Classifier model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model1 = RandomForestClassifier()
model1.fit(X_train,Y_train)
Y_pred = model1.predict(X_test)
print(accuracy_score(Y_test,Y_pred))


0.7734347976478727
